In [254]:
import pandas as pd
import numpy as np
import tqdm

In [236]:
inventories = pd.read_csv("Nuova cartella/inventories.csv")
inventory_parts = pd.read_csv("Nuova cartella/inventory_parts.csv")
colors = pd.read_csv("Nuova cartella/colors.csv")
parts = pd.read_csv("Nuova cartella/parts.csv")
part_categories = pd.read_csv("Nuova cartella/part_categories.csv")

In [233]:
inventories = inventories.replace({np.NaN:None})
inventory_parts = inventory_parts.replace({np.NaN:None})
colors = colors.replace({np.NaN:None})
parts = parts.replace({np.NaN:None})
part_categories = part_categories.replace({np.NaN:None})

## We aggragate data in acording to JsonSchema build for the inventorie collection


In [345]:
list_inventories = []

N = len(inventories)
try:
    for i in tqdm.tqdm_notebook(range(N)):
        
        dic_inventories = {} # The single document of inventories
        
        #we pull the document
        inventory_id = inventories["id"][i]
        dic_inventories["inventory_id"] = int(inventory_id)
        dic_inventories["version"] = int(inventories["version"][i])

        shallow = inventory_parts[inventory_parts["inventory_id"] == inventory_id].copy(deep = True).reset_index(drop = True)

        shallow = shallow.drop(["inventory_id"], axis = 1)
        shallow.rename(columns = {"color_id": "color"}, inplace = True)

        L = list(shallow["part_num"])

        part_cat = []
        name = []
        # whe build the array of documents (inventory_pars field)
        for el in L:
            if len(parts[parts["part_num"] == el]["part_cat_id"]) == 0:
                part_cat.append(None)
            else:
                part_cat.append(parts[parts["part_num"] == el]["part_cat_id"].iloc[0])

            if len(parts[parts["part_num"] == el]["name"]) == 0:
                name.append(None)
            else:
                name.append(parts[parts["part_num"] == el]["name"].iloc[0])

        df1 = pd.DataFrame(name, columns = ["name"]).replace({np.NaN:None})

        df2 = pd.DataFrame(part_cat, columns = ["part_cat"]).replace({np.NaN:None})


        dictionaries = pd.concat([shallow["part_num"], df1, df2, shallow[["color", "quantity", "is_spare"]]], axis=1).to_dict(orient = "records")

        for i in range(len(dictionaries)):

            val = dictionaries[i]["part_cat"]
            if val == None:
                dictionaries[i]["part_cat"] = {"part-cat_id": None, "name": None}
            else:
                dictionaries[i]["part_cat"] = part_categories[part_categories["id"] == val ].to_dict(orient = "records")[0]

            val = dictionaries[i]["color"] 
            if val == None:
                dictionaries[i]["color"] = {"color_id": None, "name": None, "rgb": None, "is_trans": None}
            else:
                dictionaries[i]["color"] = colors[colors["id"] == val ].to_dict(orient = "records")[0]

        dic_inventories["inventory_parts"] = dictionaries

        list_inventories.append(dic_inventories) # we store all the silgle dictionaries (documents) in alist
except:
    print(i)

In [ ]:
# saving a json file
with open('inventories.json', 'w') as json_file:
    json.dump(list_inventories, json_file)